In [1]:
# %pip install ftfy regex tqdm
# %pip install scikit-learn

In [2]:
import jittor as jt
from PIL import Image
import jclip as clip
import os
from tqdm import tqdm
import argparse
from sklearn.linear_model import LogisticRegression
import numpy as np
import random
# from colorama import Fore, Back, Style, init
# init()

jt.flags.use_cuda = 1
print("包导入成功")

[i 0620 18:34:19.315286 36 log.cc:351] Load log_sync: 1
[i 0620 18:34:19.372051 36 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0620 18:34:19.374432 36 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0620 18:34:19.375082 36 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xfa/IntelRXeonRPlax40/default
[i 0620 18:34:19.378082 36 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0620 18:34:19.380745 36 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0620 18:34:19.672355 36 compiler.py:1011] cuda key:cu11.7.99_sm_75
[i 0620 18:34:20.012740 36 __init__.py:227] Total mem: 30.60GB, using 10 procs for compiling.
[i 0620 18:34:20.152401 36 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0620 18:34:20.423278 36 init.cc:62] Found cuda archs: [75,]
[i 0620 18:34:23.355717 36 cuda_flags.cc:49] CUDA enabled.


包导入成功


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--split', type=str, default='A')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()
model, preprocess = clip.load("../data/ViT-B-32.pkl")
classes = open('../data/classname.txt').read().splitlines()#这是一个包含所有类的列表

## 类别文本编码,变成一句话,然后别转为向量
### 提示词可以尝试优化
text_features的shape为[374,512,]

In [4]:
# encode这块后面可以强化下
text_features=0
new_classes = []
for c in classes:
    c = c.split(' ')[0]
    if c.startswith('Animal'):
        c = c[7:]
    if c.startswith('Thu-dog'):
        c = c[8:]
    if c.startswith('Caltech-101'):
        c = c[12:]
    if c.startswith('Food-101'):
        c = c[9:]
    c = 'a photo of ' + c
    new_classes.append(c)
print(len(new_classes),"中图片")
text = clip.tokenize(new_classes)
print(text.shape)
text_features = model.encode_text(text)
print("norm前的shape",text_features.shape)
text_features /= text_features.norm(dim=-1, keepdim=True)
print(text_features.shape)
print("标签处理成功")

374 中图片
[374,77,]
norm前的shape [374,512,]
[374,512,]
标签处理成功


In [5]:
text_features.shape

[374,512,]

## 图片加载并处理

In [6]:
# training data loading
imgs_dir = '../data'
train_labels = open('../data/train.txt').read().splitlines()
train_imgs = [l.split(' ')[0] for l in train_labels]#对应的图片path
train_labels = [jt.float32([int(l.split(' ')[1])]) for l in train_labels]#对应的种类序号


Compiling Operators(36/41) used: 4.31s eta: 0.599s 39/41) used: 5.32s eta: 0.273s 41/41) used: 6.33s eta:    0s 


In [7]:
# 每个类挑四张图，根据train_labels中的label来挑选
#挑选每种的前四张,生成两个对应的列表,分别存储path和类别信息
cnt = {}
new_train_imgs = []
new_train_labels = []
for i in range(len(train_imgs)):
    label = int(train_labels[i].numpy())
    if label not in cnt:
        cnt[label] = 0
    if cnt[label] < 4:
        new_train_imgs.append(train_imgs[i])
        new_train_labels.append(train_labels[i])
        cnt[label] += 1

In [8]:
len(new_train_imgs)

1496

In [9]:
# calculate image features of training data
train_features = []
print('Training data processing:')
with jt.no_grad():
    for img in tqdm(new_train_imgs):
        img = os.path.join(imgs_dir, img)
        image = Image.open(img)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        # print("能成功运行?","image_features的shape是",image_features.shape)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        train_features.append(image_features)

train_features = jt.cat(train_features).numpy()#(1496, 512)
train_labels = jt.cat(new_train_labels).numpy()#(1496,)

Training data processing:


  5%|▍         | 70/1496 [00:01<00:37, 37.76it/s]


RuntimeError: [38;5;1m[f 0620 18:34:40.899498 36 executor.cc:686] 
Execute fused operator(2/41784) failed. 
[JIT Source]: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xfa/IntelRXeonRPlax40/default/cu11.7.99_sm_75/jit/__opkey0_broadcast_to__Tx_float32__DIM_7__BCAST_19__opkey1_reindex__Tx_float32__XDIM_4__YD___hash_763ba871e4286cdb_op.cc 
[OP TYPE]: fused_op:( broadcast_to, reindex, binary.multiply, reduce.add,)
[Input]: float32[768,3,32,32,]visual.conv1.weight, float32[1,3,224,224,], 
[Output]: float32[1,768,7,7,], 
[Async Backtrace]: not found, please set env JT_SYNC=1, trace_py_var=3 
[Reason]: [38;5;1m[f 0620 18:34:40.893989 36 helper_cuda.h:128] CUDA error at /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xfa/IntelRXeonRPlax40/default/cu11.7.99_sm_75/jit/cudnn_conv__Tx_float32__Ty_float32__Tw_float32__XFORMAT_abcd__WFORMAT_oihw__YFORMAT_abcd_____hash_4d5b3e2d24c769d3_op.cc:351  code=4( CUDNN_STATUS_INTERNAL_ERROR ) cudnnFindConvolutionForwardAlgorithm

In [ ]:
# training
classifier = LogisticRegression(random_state=0,
                                C=8.960,
                                max_iter=30000,
                                verbose=1)
classifier.fit(train_features, train_labels)

## 比赛要提交的测试

In [ ]:
# testing dataset loading
split = 'TestSet' + args.split
imgs_dir = '../data/' + split
test_imgs = os.listdir(imgs_dir)

In [ ]:
print('Testing data processing:')
test_features = []
with jt.no_grad():
    for img in tqdm(test_imgs):
        img_path = os.path.join(imgs_dir, img)
        image = Image.open(img_path)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        test_features.append(image_features)

test_features = jt.cat(test_features).numpy()

In [ ]:
# testing
with open('result_ft.txt', 'w') as save_file:
    i = 0
    predictions = classifier.predict_proba(test_features)
    for prediction in predictions.tolist():
        prediction = np.asarray(prediction)
        top5_idx = prediction.argsort()[-1:-6:-1]
        save_file.write(test_imgs[i] + ' ' +
                        ' '.join(str(idx) for idx in top5_idx) + '\n')
        i += 1